## Homework 3: Offline Policy Value Estimation (i.e. Counterfactual evaluation)

### Introduction
In this lab, we're going to be reproducing a few results from http://proceedings.mlr.press/v97/vlassis19a.html, and extending their results in a few ways.  Here's an overview: We start by taking a multiclass classification problem and splitting it into train and test.  There are 26 classes, which we'll interpret as 26 possible actions to take for every input context. On the training set, we fit a multinomial logistic regression model to predict the correct label/best action.  Following the paper, we create a logging policy based on this model (details supplied in the relevant spot below).  We then generate "logged bandit feedback" for this logging policy using the **test** set.  Given this logged bandit feedback, we'll try out several different methods for estimating the value of various policies.  We'll also estimate the value of each of these policies using the full-feedback (i.e. the full observed rewards), and we'll treat that as the ground truth value for the purpose of performance assessment.

### Load and process data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import abc

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas.api.types import is_integer_dtype
import numpy as np
from numpy.random import default_rng
from scipy.special import expit
import seaborn as sns
import warnings;
warnings.filterwarnings('ignore');
import sys
%matplotlib widget

In [3]:
def get_fully_observed_bandit():
    """
    This loads in a multiclass classification problem and reformulates it as a fully observed bandit problem.
    
    """
    df_l = pd.read_csv('data/letter-recognition.data',
                       names = ['a']+[f'x{i}' for i in range(16)])
    X = df_l.drop(columns=['a'])

    # Convert labels to ints and one-hot
    y = df_l['a']
    # if y is not column of integers (that represent classes), then convert
    if not is_integer_dtype(y.dtype):
        y = y.astype('category').cat.codes

    ## Full rewards
    n = len(y)
    k = max(y)+1
    full_rewards = np.zeros([n, k])
    full_rewards[np.arange(0,n),y] = 1
    contexts = X
    best_actions = y
    return contexts, full_rewards, best_actions

In [4]:
contexts, full_rewards, best_actions = get_fully_observed_bandit()
n, k = full_rewards.shape
_, d = contexts.shape
print(f"There are {k} actions, the context space is {d} dimensional, and there are {n} examples.")
print(f"For example, the first item has context vector:\n{contexts.iloc[0:1]}.")
print(f"The best action is {best_actions[0]}.  The reward for that action is 1 and all other actions get reward 0.")
print(f"The reward information is store in full_rewards as the row\n{full_rewards[0]}.")

There are 26 actions, the context space is 16 dimensional, and there are 20000 examples.
For example, the first item has context vector:
   x0  x1  x2  x3  x4  x5  x6  x7  x8  x9  x10  x11  x12  x13  x14  x15
0   2   8   3   5   1   8  13   0   6   6   10    8    0    8    0    8.
The best action is 19.  The reward for that action is 1 and all other actions get reward 0.
The reward information is store in full_rewards as the row
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0.].


In [ ]:
## Choose train/test indices
rng = default_rng(7)
train_frac = 0.5
train_size = round(train_frac * n)
train_idx = rng.choice(n, size = train_size, replace = False)
test_idx = np.setdiff1d(np.arange(n), train_idx, assume_unique=True)

### Policies
In this section, we'll build out a Policy class, some specific policies, and evaluate policies on full-feedback data.

**Problem 1.** Complete the Policy class and the UniformActionPolicy classes below. Run the code provided to get an estimate of the value of the uniform action policy using the test set.  Explain why the value you get makes sense.

In [ ]:
class Policy:
    def __init__(self, num_actions=2):
        self.num_actions = num_actions

    @abc.abstractmethod
    def get_action_distribution(self, X):
        """   
        This method is intended to be overridden by each implementation of Policy.

        Args:
            X (pd.DataFrame): contexts

        Returns:
            2-dim numpy array with the same number of rows as X and self.num_actions columns. 
                Each rows gives the policy's probability distribution over actions conditioned on the context in the corresponding row of X
        """   
        raise NotImplementedError("Must override method")

    def get_action_propensities(self, X, actions):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of actions
            actions (np.array): actions taken, represented by integers, corresponding to rows of X

        Returns:
            1-dim numpy array of probabilities (same size as actions) for taking each action in its corresponding context
        """   
        ## TODO
        pass

    def select_actions(self, X, rng=default_rng(1)):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of actions and propensities returned

        Returns:
            actions (np.array): 1-dim numpy array of length equal to the number of rows of X.  Each entry is an integer indicating the action selected for the corresponding context in X. 
                The action is selected randomly according to the policy, conditional on the context specified in the appropriate row of X.
            propensities (np.array): 1-dim numpy array of length equal to the number of rows of X; gives the propensity for each action selected in actions

        """   
        ## TODO
        pass
        
    def get_value_estimate(self, X, full_rewards):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of full_rewards
            full_rewards (np.array): 2-dim numpy array with the same number of rows as X and self.num_actions columns; 
                each row gives the rewards that would be received for each action for the context in the corresponding row of X.
                This would only be known in a full-feedback bandit, or estimated in a direct method

        Returns:
            scalar value giving the expected average reward received for playing the policy for contexts X and the given full_rewards

        """   
        ## TODO
        pass


class UniformActionPolicy(Policy):
    def __init__(self, num_actions=2):
        self.num_actions = num_actions

    def get_action_distribution(self, X):
        ## TODO
        pass

In [ ]:
X_train = contexts.iloc[train_idx].to_numpy()
y_train = best_actions.iloc[train_idx].to_numpy()
X_test = contexts.iloc[test_idx].to_numpy()
y_test = best_actions.iloc[test_idx].to_numpy()
full_rewards_test = full_rewards[test_idx]

uniform_policy = UniformActionPolicy(num_actions=k)
uniform_policy_value = uniform_policy.get_value_estimate(X=X_test, full_rewards=full_rewards_test)
print(f"The estimate of the value of the uniform action policy using the full-feedback test set is {uniform_policy_value}.")

**Problem 2.**  Complete the SKLearnPolicy class below and run the code that creates two policies and estimates their values using the full reward information in the test set.  You should find that the deterministic policy has a higher value than the stochastic policy.  Nevertheless, why might one choose to deploy the stochastic policy rather than the deterministic policy?

In [ ]:
class SKLearnPolicy(Policy):
    """ 
    An SKLearnPolicy uses a scikit learn model to generate an action distribution.  If the SKLearnPolicy is built with is_deterministic=False, 
    then the action distribution for a context x should be whatever predict_proba for the model returns.  If is_deterministic=True, then all the probability mass 
    should be concentrated on whatever predict of the model returns.
    """
    def __init__(self, model, num_actions=2, is_deterministic=False):
        self.is_deterministic = is_deterministic
        self.num_actions = num_actions
        self.model = model

    def get_action_distribution(self, X):
        ## TODO
        if (self.is_deterministic):
            pass
        else:
            pass

        pass

    def select_actions(self, X, rng=default_rng(1)):
        """ You don't technically have to override this function.  If you just delete this function, the parent class Policy can handle it in a generic way
        However, if is_deterministic=True, then the action distribution for each context is trivial -- it always puts probability one for a 
        particular action and 0 for the others. And so "randomly" selecting an action according to this distribution using the code you write
        for select_actions in the parent class (Policy) is very inefficient.  You can just use model.predict to get the actions that will be 
        selected for each context.  That's the idea of the if statement."""
        ## TODO
        if (self.is_deterministic):
            pass 
        else:
            pass
        
        pass

model = LogisticRegression(multi_class='multinomial')
model.fit(X_train, y_train)
policy_stochastic = SKLearnPolicy(model=model, num_actions=k, is_deterministic=False)
policy_deterministic = SKLearnPolicy(model=model, num_actions=k, is_deterministic=True)

policy_stochastic_true_value = policy_stochastic.get_value_estimate(X_test, full_rewards_test)
policy_deterministic_true_value = policy_deterministic.get_value_estimate(X_test, full_rewards_test)
print(f"Stochastic policy true value {policy_stochastic_true_value}.")
print(f"Deterministic policy true value {policy_deterministic_true_value}.")

**Problem 3.** Fill in the VlassisLoggingPolicy class below, and evaluate the value of this logging policy using the code provided.

In [ ]:
class VlassisLoggingPolicy(Policy):
    """
    This policy derives from another deterministic policy following the recipe described in the Vlassis et al paper, on the top of the second column in section 5.3.
    For any context x, if the deterministic policy selects action a, then this policy selects action a with probability eps (a supplied parameter), and spreads the
    rest of the probability mass uniformly over the other actions.
    """
    def __init__(self, deterministic_target_policy, num_actions=2, eps=0.05,):
        self.num_actions = num_actions
        self.target_policy = deterministic_target_policy
        self.eps = eps

    def get_action_distribution(self, X):
        ## TODO
        pass
    
logging_policy = VlassisLoggingPolicy(policy_deterministic, num_actions=k, eps=0.05)
logging_policy_value = logging_policy.get_value_estimate(X=X_test, full_rewards=full_rewards_test)
print(f"The estimate of the value of the logging policy using the full-feedback test set is {logging_policy_value}.")

### Simulate bandit feedback and on-policy evaluation
**Problem 4.** Take a look at the generate_bandit_feedback function, so you understand how it works.  Then generate bandit feedback using the test data -- generate as many rounds are there are contexts in the test data. Use the result to generate an "on-policy" estimate of the value of the logging policy.  How does it compare to our "ground truth" estimate you found previously using the full-feedback test set? Repeat using 1/100th, 1/10th, and 10x as much bandit feedback, to see how much the value estimates change.

In [ ]:
def generate_bandit_feedback(contexts, full_rewards, policy,
                             new_n = None,
                             rng=default_rng(1)):
    """   
    Args:
        contexts (np.array): contexts, rows correspond to entries of rewards
        full_rewards (np.array): 2-dim numpy array with the same number of rows as X and number of columns corresponding to the number actions
            each row gives the reward that would be received for each action for the context in the corresponding row of X. 

    Returns:
        new_contexts (np.array): new_n rows and same number of columns as in contexts
        actions (np.array): vector with new_n entries giving actions selected by the provided policy for the contexts in new_contexts
        observed_rewards (np.array): vector with new_n entries giving rewards received for the actions taken (in actions) in each context of new_contexts 
    """   
    
    if new_n is None:
        new_n = contexts.shape[0]
    n, k = full_rewards.shape
    num_repeats = np.ceil(new_n / n).astype(int)
    new_contexts = np.tile(contexts, [num_repeats,1])
    new_contexts = new_contexts[0:new_n]
    new_rewards = np.tile(full_rewards, [num_repeats,1])
    new_rewards = new_rewards[0:new_n]
    actions, propensities = policy.select_actions(X=new_contexts, rng=rng)
    observed_rewards = new_rewards[np.arange(new_n), actions]
    return new_contexts, actions, observed_rewards, propensities

### Test out off-policy value estimators
**Problem 5.** Complete the get_value_estimators function below, per the specification.  Include the following estimators
- Unweighted mean (done for you)
- Importance-weighted (IW) value estimator
- Self-normalized IW mean
- Direct method with linear ridge regression reward predictor fit for each action
- Direct method with IW-linear ridge regression reward predictor fit for each action
- [Optional (not for credit)] Direct method with a non-linear reward predictor fit for each action
- [Optional (not for credit)] Direct method with a non-linear reward predictor fit for all actions at once (action becomes part of the input)

Run the code below that will apply your value estimators to a policy on logged bandit feedback. Verify that your results are reasonable. (Don't worry if your numbers are not a very close match for the results in the table.)

In [ ]:
## Build our value estimators

def get_value_estimators(policy, contexts, actions, rewards, propensities, skip_slow_stuff=False):
    """   
    Args:
        policy (Policy): the policy we want to get a value estimate for
        contexts (np.array): contexts from bandit feedback
        actions (np.array): actions chosen for bandit feedback
        rewards (np.array): rewards received in bandit feedback
        propensities (np.array): the propensity for each action selected under the logging policy (which is not provided to this function)
        skip_slow_stuff (boolean): boolean flag which allows you to turn on/off some slow estimators (ignore this if you like)
    Returns:
        est (dict): keys are string describing the value estimator, values are the corresponding value estimates 
    """   

    est = {}
    est["mean"] = np.mean(rewards)
    ## TODO

    return est


In [ ]:
def get_estimator_stats(estimates, true_parameter_value=None):
    """
 
     Args:
        estimates (pd.DataFrame): each row corresponds to collection of estimates for a sample and
            each column corresponds to an estimator
        true_parameter_value (float): the true parameter value that we will be comparing estimates to
            
    Returns:
        pd.Dataframe where each row represents data about a single estimator
    """
    
    est_stat = []
    for est in estimates.columns:
        pred_means = estimates[est]
        stat = {}
        stat['stat'] = est
        stat['mean'] = np.mean(pred_means)
        stat['SD'] = np.std(pred_means)
        stat['SE'] = np.std(pred_means) / np.sqrt(len(pred_means))
        if true_parameter_value:
            stat['bias'] = stat['mean'] - true_parameter_value
            stat['RMSE'] = np.sqrt(np.mean((pred_means - true_parameter_value) ** 2))
        est_stat.append(stat)

    return pd.DataFrame(est_stat)

In [ ]:
contexts_test, actions_test, rewards_test, propensities_test = generate_bandit_feedback(contexts=X_test, full_rewards=full_rewards_test, policy=logging_policy, rng=default_rng(6))
policy = policy_deterministic
est = get_value_estimators(policy, contexts_test, actions_test, rewards_test, propensities_test)
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
print(f"policy true value {policy_true_value}.")
df = pd.DataFrame(est, index=[0])
est

**Problem 6.** Run the code below to test your value estimators across multiple trials.  Write a few sentences about anything you learned from these experiments or that you find interesting.

In [ ]:
trials=20
val_ests = []
policy = policy_deterministic
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))

In [ ]:
trials=20
val_ests = []
policy = policy_stochastic
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))

In [ ]:
trials=20
val_ests = []
policy = uniform_policy
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))